# Установка библиотеки

In [3]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=5665045bc938f6810b37177dcb54232adddd29a798fb910e701df36029e1424b
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


# Первоначальная функция, с использованием langdetect

In [4]:
from langdetect import detect

def search_english_resume(resume_json: dict) -> bool:
    """
    Определение, резюме на англиском через langdetect.
    Возвращает True, если резюме на английском.
    """
    text_fields = []

    if "title" in resume_json:
        text_fields.append(resume_json["title"])

    if "experience" in resume_json:
        for job in resume_json["experience"]:
            if "position" in job:
                text_fields.append(job["position"])
            if "description" in job and job["description"]:
                text_fields.append(job["description"])

    full_text = " ".join([t for t in text_fields if t])
    print(full_text)
    if not full_text.strip():
        print(False)

    try:
        return detect(full_text) != "ru"
    except:
        return False

In [7]:
resumes_json = [{
  'id': '685e5a6a000c57368900000d3c44314e786d4e',
  'last_name': None,
 'first_name': None,
 'middle_name': None,
 'title': 'Начинающий специалист', #'Начинающий специалист A novice specialist'
 'experience': [{'start': '2023-06-02',
                 'end': '2023-08-02',
                 'company': 'Smoky Bro',
                 'company_id': '1898857',
                 'industry': None,
                 'industries': [],
                 'area': None, 'company_url': None,
                 'employer': {'id': '4839077', 'name': 'Smoky Bro', 'url': 'https://api.hh.ru/employers/4839077', 'alternate_url': 'https://hh.ru/employer/4839077', 'logo_urls': None},
                 'position': 'Продавец',
                 'description': 'Тестовое поле описания опыта'}],  #Тестовое поле описания опыта The test field of the experience description
  },
  {
  'id': '685e5a6a000cgkgyuul6d4e',
  'last_name': None,
  'first_name': None,
  'middle_name': None,
  'title': 'A novice specialist', #'Начинающий специалист A novice specialist'
 'experience': [{'start': '2023-06-02',
                 'end': '2023-08-02',
                 'company': 'Smoky Bro',
                 'company_id': '1898857',
                 'industry': None,
                 'industries': [],
                 'area': None, 'company_url': None,
                 'employer': {'id': '4839077', 'name': 'Smoky Bro', 'url': 'https://api.hh.ru/employers/4839077', 'alternate_url': 'https://hh.ru/employer/4839077', 'logo_urls': None},
                 'position': 'Продавец',
                 'description': 'The test field of the experience description'}],  #Тестовое поле описания опыта The test field of the experience description
  }
              ]

In [8]:
for resume in resumes_json:
    if search_english_resume(resume):
        print(f"skip EN resume {resume['id']}")
        #continue


Начинающий специалист Продавец Тестовое поле описания опыта
A novice specialist Продавец The test field of the experience description
skip EN resume 685e5a6a000cgkgyuul6d4e


# Итоговая функция, с использованием регулярок

In [9]:
import re

text = "Менеджер Manager й q e ,fhvty бармен"
print(re.findall(r"[а-яё]", text.lower())) # Разбивает предложение на буквы
print(re.findall(r"[a-z]", text.lower()))

['м', 'е', 'н', 'е', 'д', 'ж', 'е', 'р', 'й', 'б', 'а', 'р', 'м', 'е', 'н']
['m', 'a', 'n', 'a', 'g', 'e', 'r', 'q', 'e', 'f', 'h', 'v', 't', 'y']


In [23]:
import re

def detect_language_by_alphabet(text: str) -> str:
    """
    Определяем язык текста по количеству символов кириллицы и латиницы.
    Возвращает 'ru', 'en' или 'unknown'.
    """
    text = text.lower()
    count_rus = len(re.findall(r"[а-яё]", text))
    count_eng = len(re.findall(r"[a-z]", text))
    if count_rus > count_eng:
        return "rus"
    elif count_eng > count_rus:
        return "eng"
    else:
        return "unknown"


def is_english_resume(resume_json: dict) -> bool:
    """
    Определение, резюме на английском.
    Возвращает True, если латинских букв больше, чем кириллических.
    """
    text_fields = []

    if "title" in resume_json and resume_json["title"]:
        text_fields.append(resume_json["title"])

    if "experience" in resume_json:
        for job in resume_json["experience"]:
            if "position" in job and job["position"]:
                text_fields.append(job["position"])
            if "description" in job and job["description"]:
                text_fields.append(job["description"])

    full_text = " ".join([t for t in text_fields if t])

    if not full_text.strip():
        return False

    lang = detect_language_by_alphabet(full_text)
    if lang == "eng":
      print('test: ', full_text)
    return lang == "eng"

In [24]:
resumes_test = [
    {
        'id': '1',
        'title': 'Начинающий специалист',
        'experience': [
            {'position': 'Продавец', 'description': 'Тестовое поле описания опыта'}
        ]
    },
    {
        'id': '2',
        'title': 'Junior Specialist',
        'experience': [
            {'position': 'Sales Manager', 'description': 'Worked with clients and increased sales'}
        ]
    },
    {
        'id': '3',
        'title': 'Менеджер Manager',
        'experience': [
            {'position': 'Administrator', 'description': 'Работал с документацией'}
        ]
    },
    {
        'id': '4',
        'title': 'Администратор',
        'experience': []
    },
    {
        'id': '5',
        'title': '',
        'experience': []
    },
    {
        'id': '5',
        'title': 'Бармен',
        'experience': []
    },
    {
    "id": "6",
    "title": "Старший аналитик данных",
    "experience": [
        {
            "position": "Аналитик данных",
            "description": (
                "Разрабатывал и поддерживал системы сбора и обработки больших данных с использованием Apache Spark и Airflow. "
                "Оптимизировал SQL-запросы и создавал ETL-процессы для аналитических систем и моделей машинного обучения. "
                "Работал в кросс-функциональных командах, участвовал в проектировании архитектуры данных, "
                "обеспечивая высокую доступность и масштабируемость. "
                "Имеется опыт работы с облачными платформами (AWS, GCP) и автоматизацией CI/CD."
            )
        },
        {
            "position": "Разработчик Python",
            "description": (
                "Создавал внутренние сервисы и API на Python и FastAPI. "
                "Улучшил производительность приложений на 30% за счёт оптимизации и рефакторинга кода. "
                "Реализовал автоматическое тестирование, интегрировал системы логирования и мониторинга."
            )
        }
    ]
},
  {
    "id": "7",
    "title": "Senior Data Engineer",
    "experience": [
        {
            "position": "Data Engineer",
            "description": (
                "Developed and maintained large-scale data pipelines using Apache Spark and Airflow. "
                "Worked with distributed systems, optimized SQL queries, and implemented ETL processes "
                "to support analytics and machine learning models. Collaborated with cross-functional teams "
                "to design data architectures, ensuring high availability and scalability. "
                "Experience in cloud platforms (AWS, GCP) and CI/CD automation."
            )
        },
        {
            "position": "Python Developer",
            "description": (
                "Built internal tools and APIs using Python and FastAPI. "
                "Improved application performance by 30% through code refactoring and optimization. "
                "Implemented automated testing pipelines and integrated logging/monitoring systems."
            )
        }
    ]
}
]


In [25]:
for resume in resumes_test:
    if is_english_resume(resume):
        print(f"skip EN resume {resume['id']}")
    else:
        print(f"accept RU resume {resume['id']}")


accept RU resume 1
test:  Junior Specialist Sales Manager Worked with clients and increased sales
skip EN resume 2
accept RU resume 3
accept RU resume 4
accept RU resume 5
accept RU resume 5
accept RU resume 6
test:  Senior Data Engineer Data Engineer Developed and maintained large-scale data pipelines using Apache Spark and Airflow. Worked with distributed systems, optimized SQL queries, and implemented ETL processes to support analytics and machine learning models. Collaborated with cross-functional teams to design data architectures, ensuring high availability and scalability. Experience in cloud platforms (AWS, GCP) and CI/CD automation. Python Developer Built internal tools and APIs using Python and FastAPI. Improved application performance by 30% through code refactoring and optimization. Implemented automated testing pipelines and integrated logging/monitoring systems.
skip EN resume 7
